# EDA for Train.csv

# 1. 작업 환경 설정 및 데이터 구조 확인

In [97]:
import pandas as pd
import hvplot.pandas

In [98]:
# data load
train = pd.read_csv('../input/train.csv')

In [99]:
# data 정보 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201917 entries, 0 to 201916
Data columns (total 6 columns):
first_active_month    201917 non-null object
card_id               201917 non-null object
feature_1             201917 non-null int64
feature_2             201917 non-null int64
feature_3             201917 non-null int64
target                201917 non-null float64
dtypes: float64(1), int64(3), object(2)
memory usage: 9.2+ MB


In [100]:
# 결측치 개수 확인
train.isnull().sum()

first_active_month    0
card_id               0
feature_1             0
feature_2             0
feature_3             0
target                0
dtype: int64

## 2. first_active_month

In [104]:
# 각 first_active_month의 개수 확인
first_active_month_count = pd.DataFrame(train['first_active_month'].value_counts()).reset_index()
first_active_month_count.rename(columns = {"first_active_month":'count', 'index':'first_active_month'}, inplace=True)

# first_active_month를 기준으로 오름차순으로 정렬 후 시각화
sorted_first_active_month_count = first_active_month_count.sort_values('first_active_month')
sorted_first_active_month_count.hvplot.line(x='first_active_month', y='count', rot=90, width=800, title="시간에 따른 first_active_month의 분포")

:Curve   [first_active_month]   (count)

- 전체적으로 보았을 때 지속적으로 증가하다 2017년 9월 정점을 찍은 후 급격히 감소했다.
- 첫 몇년간은 완만하게 증가했다.
- 특정 시점에서 갑작스럽게 증가하거나 감소하는 등 급격한 변화가 있었다.

In [103]:
# Q. 직전달과 비교했을 때 변화율이 가장 높은 구간은?
sorted_first_active_month_count['pct_change'] = sorted_first_active_month_count['count'].pct_change()
sorted_first_active_month_count.sort_values('pct_change').head()

,first_active_month,count,pct_change
62,2018-01,34,-0.996657
74,2018-02,1,-0.970588
73,2011-12,2,-0.750000
68,2012-12,18,-0.625000
67,2012-06,19,-0.321429


- 직전달과 비교했을 때 2018년 1월과 2월의 변화율이 각각 99%, 98%로 가장 컸다.